In [10]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [11]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/riadanas/Desktop/housing regression MLE/.venv/bin/python
3.0.4
/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [12]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

In [13]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (585244, 39)
Eval shape: (149424, 39)


In [14]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [15]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/Users/riadanas/Desktop/housing regression MLE/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

[I 2025-08-30 10:57:33,987] A new study created in memory with name: no-name-2f0eef7f-cbf8-47e1-be86-7008ebe9e609
[I 2025-08-30 10:57:38,628] Trial 0 finished with value: 70146.8484159042 and parameters: {'n_estimators': 288, 'max_depth': 8, 'learning_rate': 0.09363331563780464, 'subsample': 0.5164554883425666, 'colsample_bytree': 0.622497141242759, 'min_child_weight': 8, 'gamma': 2.7167518381831335, 'reg_alpha': 1.8783162156492189e-07, 'reg_lambda': 2.1114076262158873e-06}. Best is trial 0 with value: 70146.8484159042.
[I 2025-08-30 10:57:47,565] Trial 1 finished with value: 78339.84815630759 and parameters: {'n_estimators': 933, 'max_depth': 5, 'learning_rate': 0.010525395887875854, 'subsample': 0.9451395386785079, 'colsample_bytree': 0.8649757658432999, 'min_child_weight': 9, 'gamma': 0.4193862975691942, 'reg_alpha': 0.1715414628379197, 'reg_lambda': 2.45067528730062e-07}. Best is trial 0 with value: 70146.8484159042.
[I 2025-08-30 10:57:51,985] Trial 2 finished with value: 76210.01

Best params: {'n_estimators': 352, 'max_depth': 9, 'learning_rate': 0.08965741427766928, 'subsample': 0.5044884308630853, 'colsample_bytree': 0.6845211350293073, 'min_child_weight': 1, 'gamma': 1.5777909383735702, 'reg_alpha': 5.700889859752827e-06, 'reg_lambda': 3.6122262314823315e-08}


In [16]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32111.79066601309
RMSE: 74805.68751722854
R²: 0.9567001901792198


/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [10:59:35] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/08/30 10:59:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/30 10:59:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
